In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git

In [1]:
import pandas as pd

# **Mistral-7B Results**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.nn import DataParallel


tokenizer = AutoTokenizer.from_pretrained('NousResearch/Yarn-Mistral-7b-128k')
model = AutoModelForCausalLM.from_pretrained('NousResearch/Yarn-Mistral-7b-128k')
print(model)

**Mistal-7B (Zero-shot)**

In [ ]:
df = pd.read_csv('/content/Test_modified_432.csv')

llm_generated_templates = []

for index, row in df.iterrows():
    log_event = row['Content']
    ground_truth = row['EventTemplate']


    prompt = f"""
<|im_start|>system
You are a log parsing assistant. Your task is to identify variable elements within the logs, generalize these elements, and construct a template that represents the structure of these log messages.
<|im_end|>

<|im_start|>user
Extract the template for the following log message. Replace any variable element with the placeholder '<*>'. Do not provide any explanation, only return the template.
```{log_event}```
<|im_end|>

<|im_start|>assistant
    """
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=60,
        )[0],
        skip_special_tokens=True
    )
    print("========================================")
    print(f"index: {index}")
    print(f"Whole Output: {output}")
    print("========================================")

    output = output.replace('"', '')
    output = output.replace('\n', '')

    after_assistant = output.split("assistant")[-1]

    if "<|im_end|>" in after_assistant:
        parsed_output = after_assistant.split("<|im_end|>")[0].strip()
    else:
        parsed_output = after_assistant.strip()

    llm_generated_templates.append(parsed_output)
    print("----------------------------------------------")
    print(f"Log Event: {log_event}")
    print(f"Ground Truth: {ground_truth}")
    print(f"Extracted Template: {parsed_output}")
    print("----------------------------------------------")

df['ZsMistralExtractedTemplate'] = llm_generated_templates
df.to_csv('/content/ZS_Test_modified_432.csv', index=False)

**Mistral-7B (Few-shot)**

In [ ]:
df = pd.read_csv('/content/ZS_Test_modified_432.csv')

llm_generated_templates = []

for index, row in df.iterrows():
    log_event = row['Content']
    ground_truth = row['EventTemplate']
    # log_event = row['Content'].strip('"')
    # ground_truth = row['EventTemplate'].strip('"')

    prompt = f"""
<|im_start|>system
You are a log parsing assistant. Your task is to identify variable elements within the logs, generalize these elements, and construct a template that represents the structure of these log messages.
<|im_end|>

<|im_start|>user
For your reference, here are a few log message examples and their corresponding templates.

Example 1
Log message: ```Error = , LOGIN chdir(/home/spelce1/UMT2K/umt2k/ckpt_umt2k_src/TEST/NEW_TEST) failed: No such file at /10.10.34.20:56374 point [/TEST/NEW_TEST], connect to proxy proxy.cse.cuhk.edu.hk:5070 to renew session (0x14f05578bd8001b)```
Extracted template: ```Error = , LOGIN chdir(<*>) failed: No such file at <*>:<*> point [<*>], connect to proxy <*>:<*> to renew session (<*>)```

Example 2
Log message: ```ciod: In packet from nodes 91.0 and node-234 (R62-M1-Nf-C:J03-U11), message code 2 is not 3 or 4294967295 (softheader=003b005b 00030000 00000001 00000000)```
Extracted template: ```ciod: In packet from nodes <*> and <*> (<*>:<*>), message code <*> is not <*> or <*> (softheader=<*> <*> <*> <*>)```

Now, extract the template for the following log message. Replace any variable element with the placeholder '<*>'. Do not provide any explanation, only return the template.
```{log_event}```
<|im_end|>

<|im_start|>assistant
    """
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=60,
        )[0],
        skip_special_tokens=True
    )
    print("========================================")
    print(f"index: {index}")
    print(f"Whole Output: {output}")
    print("========================================")

    output = output.replace('"', '')
    output = output.replace('\n', '')

    after_assistant = output.split("assistant")[-1]

    if "<|im_end|>" in after_assistant:
        parsed_output = after_assistant.split("<|im_end|>")[0].strip()
    else:
        parsed_output = after_assistant.strip()

    llm_generated_templates.append(parsed_output)
    print("----------------------------------------------")
    print(f"Log Event: {log_event}")
    print(f"Ground Truth: {ground_truth}")
    print(f"Extracted Template: {parsed_output}")
    print("----------------------------------------------")

df['FsMistralExtractedTemplate'] = llm_generated_templates
df.to_csv('/content/FS_ZS_Test_modified_432.csv', index=False)

**Mistal-7B (Fine-tuned)**



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.nn import DataParallel


tokenizer = AutoTokenizer.from_pretrained('unknown/Yarn-Mistral-7b-128k_Fine-Tuned4LogParsing-r2')
model = AutoModelForCausalLM.from_pretrained('unknown/Yarn-Mistral-7b-128k_Fine-Tuned4LogParsing-r2')
print(model)

In [ ]:
df = pd.read_csv('/content/FS_ZS_Test_modified_432.csv')

llm_generated_templates = []

for index, row in df.iterrows():
    log_event = row['Content']
    ground_truth = row['EventTemplate']
    # log_event = row['Content'].strip('"')
    # ground_truth = row['EventTemplate'].strip('"')

    prompt = f"""
<|im_start|>system
You are a log parsing assistant. Your task is to identify variable elements within the logs, generalize these elements, and construct a template that represents the structure of these log messages.
<|im_end|>

<|im_start|>user
Extract the template for the following log message. Replace any variable element with the placeholder '<*>'. Do not provide any explanation, only return the template.
```{log_event}```
<|im_end|>

<|im_start|>assistant
    """
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=60,
        )[0],
        skip_special_tokens=True
    )
    print("========================================")
    print(f"index: {index}")
    print(f"Whole Output: {output}")
    print("========================================")

    output = output.replace('"', '')
    output = output.replace('\n', '')

    after_assistant = output.split("assistant")[-1]

    if "<|im_end|>" in after_assistant:
        parsed_output = after_assistant.split("<|im_end|>")[0].strip()
    else:
        parsed_output = after_assistant.strip()

    llm_generated_templates.append(parsed_output)
    print("----------------------------------------------")
    print(f"Log Event: {log_event}")
    print(f"Ground Truth: {ground_truth}")
    print(f"Extracted Template: {parsed_output}")
    print("----------------------------------------------")

df['FTMistralExtractedTemplate'] = llm_generated_templates
df.to_csv('/content/FT_FS_ZS_Test_modified_432.csv', index=False)

# **GPT4 Results**

In [ ]:
!pip install openai

In [ ]:
!pip install langchain

In [ ]:
import os
import openai
import random
import pandas as pd

**GPT-4 (Zero-shot)**

In [ ]:
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

def gpt_reponse(log_event, ground_truth):
  # openai_api_key = ''

  openai_api_key = ''

  gpt4_prompt = """
You are a log parsing assistant. Your task is to identify variable elements within the logs, generalize these elements, and construct a template that represents the structure of these log messages.

Extract the template for the following log message. Replace any variable element with the placeholder '<*>'. Do not provide any explanation, only return the template.
```{log_event}```
  """

  prompt = PromptTemplate(
    input_variables = ['log_event'],
    template = gpt4_prompt
    )

  prompt.format(log_event = log_event)

  # llm = OpenAI(temperature=0, openai_api_key = openai_api_key)

  llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key = openai_api_key)
  chain = LLMChain(llm = llm, prompt = prompt)
  response = chain({'log_event': log_event})
  # print(response)
  extracted_template = response.get('text', 'No Values')
  # extracted_template = extracted_template.strip('```');
  print('==================================================================')

  print (f"Log Event: {log_event}")
  print (f"Ground Truth: {ground_truth}")
  print (f"Extracted Template: {extracted_template}")


  return extracted_template



# csv_file_path = '/content/samp.csv'
csv_file_path = '/content/Test_modified_432.csv'
df = pd.read_csv(csv_file_path)

df['ZSGPT4ExtractedTemplate'] = df.apply(lambda row: gpt_reponse(row['Content'], row['EventTemplate']), axis=1)
# df.to_csv('/content/GPT4Zh_FT_FS_ZS_Test_modified_432.csv', index=False)
df.to_csv('/content/GPT4ZS_Test_modified_432.csv', index=False)

print('==================================================================')
print("Results added to the CSV file.")


**GPT-4 (Few-shot)**

In [ ]:
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

def gpt_reponse(log_event, ground_truth):
  # openai_api_key = ''

  openai_api_key = ''

  gpt4_prompt = """
You are a log parsing assistant. Your task is to identify variable elements within the logs, generalize these elements, and construct a template that represents the structure of these log messages.

For your reference, here are a few log message examples and their corresponding templates.

Example 1
Log message: ```Error = , LOGIN chdir(/home/spelce1/UMT2K/umt2k/ckpt_umt2k_src/TEST/NEW_TEST) failed: No such file at /10.10.34.20:56374 point [/TEST/NEW_TEST], connect to proxy proxy.cse.cuhk.edu.hk:5070 to renew session (0x14f05578bd8001b)```
Extracted template: ```Error = , LOGIN chdir(<*>) failed: No such file at <*>:<*> point [<*>], connect to proxy <*>:<*> to renew session (<*>)```

Example 2
Log message: ```ciod: In packet from nodes 91.0 and node-234 (R62-M1-Nf-C:J03-U11), message code 2 is not 3 or 4294967295 (softheader=003b005b 00030000 00000001 00000000)```
Extracted template: ```ciod: In packet from nodes <*> and <*> (<*>:<*>), message code <*> is not <*> or <*> (softheader=<*> <*> <*> <*>)```

Now, extract the template for the following log message. Replace any variable element with the placeholder '<*>'. Do not provide any explanation, only return the template.
```{log_event}```
  """

  prompt = PromptTemplate(
    input_variables = ['log_event'],
    template = gpt4_prompt
    )

  prompt.format(log_event = log_event)

  # llm = OpenAI(temperature=0, openai_api_key = openai_api_key)
  llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key = openai_api_key)
  chain = LLMChain(llm = llm, prompt = prompt)
  response = chain({'log_event': log_event})

  extracted_template = response.get('text', 'No Values')

  print('==================================================================')

  print (f"Log Event: {log_event}")
  print (f"Ground Truth: {ground_truth}")
  print (f"Extracted Template: {extracted_template}")


  return extracted_template



#csv_file_path = '/content/evaltest.csv'
csv_file_path = '/content/Test_modified_432.csv'
df = pd.read_csv(csv_file_path)

df['FSGPT4ExtractedTemplate'] = df.apply(lambda row: gpt_reponse(row['Content'], row['EventTemplate']), axis=1)
df.to_csv('/content/GPT4FS_Test_modified_432.csv', index=False)

print('==================================================================')
print("Results added to the CSV file.")


# **GPT4 as Evaluator**

In [ ]:
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
import re
import time

def evaluator(content, event_template, generated_template):
  # openai_api_key = ''

  c= content
  et = event_template
  gt = generated_template

  openai_api_key = ''

  eval_prompt = """
Task Description:
Log parsing is the process of converting unstructured log messages into structured event templates by extracting the constant and variable parts of raw logs.
You are a log parsing evaluator that assesses the quality of an extracted log template from a log message, given a ground truth template.

Evaluation Criteria:
Your evaluation must cover the accuracy and robustness of the extracted template.
- Accuracy: This gauges the fidelity of the extracted templates in capturing the constant and variable parts of log messages, as defined by the ground truth.
Accuracy is measured by the degree to which the extracted template aligns with the ground truth.
- Robustness: This gauges the adaptability of the extracted templates across varied log types, including those with noise, inconsistencies, or diverse structures.
Robustness is measured by the degree to which the extracted template is capable of preserving its quality and representation across diverse logs.

Scoring:
You will explain your reasoning and give a score between 0 and 5 for each evaluation criterion.
A score of 0 means the extracted template is completely wrong or unusable, while a score of 5 means the extracted template is perfect or optimal.
Assign a score of 5 for both Accuracy and Robustness if the extracted template matches the ground truth template exactly.

Examples:
For your reference, here are a few evaluation examples.

- Example 1
Log message: ```Client attempting to renew session (0x14f05578bd8001b) at (/10.10.34.20:56374)```
Extracted template: ```Client attempting to renew session <*> at <*>```
Ground truth template: ```Client attempting to renew session (<*>) at (<*>:<*>)```

Evaluation:
- Accuracy: The extracted template simplifies the variable parts into two placeholders <*>, which corresponds to the session ID and the IP:port pair in the log message.
  However, it does not accurately reflect the structure of the variable parts as indicated in the ground truth template. The ground truth specifies three separate placeholders
  within parentheses and a distinct separation for the IP address and port, suggesting a more granular level of detail. Accuracy score: 3.
- Robustness: The extracted template's simplified approach to variable parts (<*> at <*>) could be seen as more adaptable to a wider range of log messages, as it does not strictly enforce the format of the variable parts.
  This could potentially make the template more robust when dealing with logs that have minor deviations from the expected format, such as missing parentheses or variations in how the session ID and the IP:port pair are presented. Robustness score: 5.

- Example 2
Log message: ```Unknown base file: /var/lib/nova/instances/_base/a489c868f0c37da93b76227c91bb03908ac0e742```
Extracted template: ```Unknown base file: /var/lib/nova/instances/_base/<*>```
Ground truth template: ```Unknown base file: <*>```

Evaluation:
- Accuracy: The extracted template accurately represents the constant part of the log message up to /var/lib/nova/instances/_base/, and correctly identifies the variable part as <*>.
  However, it does not fully align with the ground truth's approach of considering the entire path as variable.  Accuracy score: 4.
- Robustness: The extracted template specifies a particular path structure (/var/lib/nova/instances/_base/) before introducing the variable component.
  This specificity might limit its adaptability across varied log messages that do not follow this exact path structure but still pertain to unknown base files. Robustness score: 3.

Now, evaluate the accuracy and robustness of the extracted template for the following log message given the ground truth template. Explain your reasoning and give a score between 0 and 5 for each evaluation criterion.
Log message: ```{log_event}```
Extracted template: ```{generated_template}```
Ground truth template: ```{ground_truth}```
"""

  prompt = PromptTemplate(
    input_variables = ['log_event', 'ground_truth', 'generated_template'],
    template = eval_prompt
    )

  prompt.format(log_event = content, ground_truth = event_template, generated_template = generated_template)

  #llm = OpenAI(temperature=0, openai_api_key = openai_api_key)
  llm = ChatOpenAI(model="gpt-4", temperature=0,openai_api_key = openai_api_key)

  # print(llm.model_name)
  print('---------------------------------------------------')
  chain = LLMChain(llm = llm, prompt = prompt)
  response = chain({'log_event': content, 'ground_truth': event_template, 'generated_template': generated_template})
  # print (f"Whole response: {response}")

  metrics = response.get('text', 'No Values')


  accuracy_match = re.search(r"Accuracy score: (\d+(\.\d+)?)\.", metrics)
  accuracy_score = accuracy_match.group(1) if accuracy_match else None

  accuracy_description_match = re.search(r"- Accuracy: (.*?) Accuracy score:", metrics, re.DOTALL)
  accuracy_description = accuracy_description_match.group(1).strip() if accuracy_description_match else None



  robustness_match = re.search(r"Robustness score: (\d+(\.\d+)?)\.", metrics)
  robustness_score = robustness_match.group(1) if robustness_match else None

  robustness_description_match = re.search(r"- Robustness: (.*?) Robustness score:", metrics, re.DOTALL)
  robustness_description = robustness_description_match.group(1).strip() if robustness_description_match else None


  print(f"Accuracy score: {accuracy_score}")
  print(f"Accuracy reasoning: {accuracy_description}")
  print(f"Robustness score: {robustness_score}")
  print(f"Robustness reasoning: {robustness_description}")
  #print(f"Score: {score}")

  return accuracy_score, accuracy_description, robustness_score, robustness_description



input_csv_file_path = '/content/GPT4EvalZSMistral.csv'
df = pd.read_csv(input_csv_file_path)

evaluation_results = []
for index, row in df.iterrows():
    print(index)
    result = evaluator(row['Content'], row['EventTemplate'], row['ZsMistralExtractedTemplate'])
    # accuracy_score, accuracy_description, robustness_score, robustness_description = result
    # if accuracy_score is None or accuracy_description is None or robustness_score is None or robustness_description is None:
      # print('None')
    # result = evaluator(row['Content'], row['EventTemplate'], row['ZsMistralExtractedTemplate'])
    # else:
    evaluation_results.append(result)

    # After processing each row, wait for 20 seconds before continuing to the next one
    # This limits the script to 3 requests per minute
    # time.sleep(30)

df[['ZSMist_Acc', 'ZSMist_Acc_despn', 'ZSMist_Robust', 'ZSMist_Robust_despn']] = pd.DataFrame(evaluation_results, index=df.index)

output_csv_file_path = '/content/GPT4EvalZSMistral.csv'
df.to_csv(output_csv_file_path, index=False)
